In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import seaborn as sns
%matplotlib inline

In [2]:
# Read in City Summary.csv
testing = pd.read_csv("city_summary_data_results.csv")
testing.head()

,Unnamed: 0,Unnamed: 1,Month,Avg AQI_15,Avg AQI_16,Avg AQI_17,Avg AQI_18,Avg AQI_19,Avg AQI_20,Unnamed: 0.1
0,Atlanta,0,1,123.0,130.0,118.0,114.0,101.0,87.0,NaN
1,Atlanta,1,2,123.0,124.0,92.0,99.0,98.0,89.0,NaN
2,Atlanta,2,3,105.0,130.0,90.0,113.0,109.0,107.0,NaN
3,Atlanta,3,4,87.0,121.0,80.0,99.0,100.0,96.0,NaN
4,Atlanta,4,5,130.0,101.0,57.0,107.0,125.0,87.0,NaN


In [3]:
# Rename columns
testing2 = testing.rename(columns={"Unnamed: 0":"City"})
testing2.head()

,City,Unnamed: 1,Month,Avg AQI_15,Avg AQI_16,Avg AQI_17,Avg AQI_18,Avg AQI_19,Avg AQI_20,Unnamed: 0.1
0,Atlanta,0,1,123.0,130.0,118.0,114.0,101.0,87.0,NaN
1,Atlanta,1,2,123.0,124.0,92.0,99.0,98.0,89.0,NaN
2,Atlanta,2,3,105.0,130.0,90.0,113.0,109.0,107.0,NaN
3,Atlanta,3,4,87.0,121.0,80.0,99.0,100.0,96.0,NaN
4,Atlanta,4,5,130.0,101.0,57.0,107.0,125.0,87.0,NaN


In [4]:
# Groupby City and calculate the mean of each year
summary = testing2.groupby(["City"])

avg_2018 = round(summary["Avg AQI_18"].mean(),2)
avg_2019 = round(summary["Avg AQI_19"].mean(),2)
avg_2020 = round(summary["Avg AQI_20"].mean(),2)

In [5]:
# Create new dataframe
avg_aqi_cities = pd.DataFrame({
                            "Avg AQI 2018":avg_2018,
                            "Avg AQI 2019":avg_2019,
                            "Avg AQI 2020":avg_2020
    
    
                        })

In [6]:
# Reset index so City is a column again
cities_df = avg_aqi_cities.reset_index()

In [7]:
# Import dependencies for Google APIs
import requests
import json

# Google API Key
from api_keys import g_key

In [8]:
# Create Latitude and Longitude columns
cities_df["Lat"]=""
cities_df["Lng"]=""

In [9]:
# Display dataframe with new columns
cities_df.head()

,City,Avg AQI 2018,Avg AQI 2019,Avg AQI 2020,Lat,Lng
0,Atlanta,107.29,108.14,95.86,,
1,Fresno,112.43,96.57,102.29,,
2,Los Angeles,137.14,121.29,119.00,,
3,Melbourne,101.64,90.10,91.15,,
4,Mexico City,158.00,158.57,152.57,,


In [10]:
# Grabbing the latitude and longitude from Google Geocode API
params = {"key":g_key}

for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    city = row["City"]
    
    
    params['address'] = f"{city}"
    
    cities_lat_lng = requests.get(base_url,params=params)
    
    cities_lat_lng = cities_lat_lng.json()
    
    cities_df.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
  
    cities_df

In [12]:
# Setting the city as the index to use .loc
cities_pd = cities_df.set_index("City")
cities_pd

,Avg AQI 2018,Avg AQI 2019,Avg AQI 2020,Lat,Lng
City,,,,,
Atlanta,107.29,108.14,95.86,33.749,-84.388
Fresno,112.43,96.57,102.29,36.7378,-119.787
Los Angeles,137.14,121.29,119.00,34.0522,-118.244
Melbourne,101.64,90.10,91.15,-37.8136,144.963
Mexico City,158.00,158.57,152.57,19.4326,-99.1332
Miami,99.83,NaN,100.00,25.7617,-80.1918
Mumbai,195.75,162.29,151.71,19.076,72.8777
New Dehli,223.14,220.71,200.57,28.6139,77.209
New York City,108.00,102.86,94.00,40.7128,-74.006


## North and Central America

In [13]:
# Import gmaps for heatmap
import gmaps

# Configure gmaps
gmaps.configure(api_key=g_key)

In [14]:
# Locating North and Central American cities for new dataframe
north_america = cities_pd.loc[["Atlanta","Fresno","Los Angeles","Mexico City","Miami","New York City","San Francisco"],
                              ["Avg AQI 2018","Avg AQI 2019","Avg AQI 2020","Lat","Lng"]]

north_america

,Avg AQI 2018,Avg AQI 2019,Avg AQI 2020,Lat,Lng
City,,,,,
Atlanta,107.29,108.14,95.86,33.749,-84.388
Fresno,112.43,96.57,102.29,36.7378,-119.787
Los Angeles,137.14,121.29,119.00,34.0522,-118.244
Mexico City,158.00,158.57,152.57,19.4326,-99.1332
Miami,99.83,NaN,100.00,25.7617,-80.1918
New York City,108.00,102.86,94.00,40.7128,-74.006
San Francisco,98.71,90.00,88.86,37.7749,-122.419


In [30]:
# Resetting the index so City can be a column again
nc_america = north_america.reset_index()

In [31]:
# Display North-Central America dataframe for heatmap
nc_america

,City,Avg AQI 2018,Avg AQI 2019,Avg AQI 2020,Lat,Lng
0,Atlanta,107.29,108.14,95.86,33.749,-84.388
1,Fresno,112.43,96.57,102.29,36.7378,-119.787
2,Los Angeles,137.14,121.29,119.00,34.0522,-118.244
3,Mexico City,158.00,158.57,152.57,19.4326,-99.1332
4,Miami,99.83,NaN,100.00,25.7617,-80.1918
5,New York City,108.00,102.86,94.00,40.7128,-74.006
6,San Francisco,98.71,90.00,88.86,37.7749,-122.419


In [41]:
locations = nc_america[["Lat","Lng"]]

# Added below in comments to switch out years fairly easily
# aqi_15 = nc_america["Avg AQI 2015"].astype(float)
# aqi_16 = nc_america["Avg AQI 2016"].astype(float)
# aqi_17 = nc_america["Avg AQI 2017"].astype(float)
# aqi_18 = nc_america["Avg AQI 2018"].astype(float)
# aqi_19 = nc_america["Avg AQI 2019"].astype(float)   
aqi_20 = nc_america["Avg AQI 2020"].astype(float)

In [42]:
# North and Central America heatmap with AQI info
fig = gmaps.figure(map_type="TERRAIN",center=(29.749907,-95.358421),zoom_level=4)

heat_layer = gmaps.heatmap_layer(locations, weights=aqi_20,dissipating=False,max_intensity=200,
                               point_radius=5)


info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>AQI 2020<dt><dd>{Avg AQI 2020}</dd>
</dl>
"""

city_info = [info_box_template.format(**row) for index, row in nc_america.iterrows()]
locations = nc_america[["Lat","Lng"]] 

symbols = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red',info_box_content=city_info)

                         
                         
fig.add_layer(heat_layer)
fig.add_layer(symbols)
fig


Figure(layout=FigureLayout(height='420px'))

## Europe

In [43]:
# Pulling the cities of Europe for heatmap

europe = cities_pd.loc[["Rome","Stockholm"],
                              ["Avg AQI 2018","Avg AQI 2019","Avg AQI 2020","Lat","Lng"]]


In [44]:
# Reset Index
europe_df = europe.reset_index()
europe_df

,City,Avg AQI 2018,Avg AQI 2019,Avg AQI 2020,Lat,Lng
0,Rome,116.03,121.01,117.11,41.9028,12.4964
1,Stockholm,95.91,90.07,67.68,59.3293,18.0686


In [50]:
# Set locations
locations = europe_df[["Lat","Lng"]]
# aqi_15 = europe_df["Avg AQI 2015"].astype(float)
# aqi_16 = europe_df["Avg AQI 2016"].astype(float)
# aqi_17 = europe_df["Avg AQI 2017"].astype(float)
# aqi_18 = europe_df["Avg AQI 2018"].astype(float)
# aqi_19 = europe_df["Avg AQI 2019"].astype(float)   
aqi_20 = europe_df["Avg AQI 2020"].astype(float)

In [51]:
# Heatmap for Europe
#,center=(29.749907,-95.358421),zoom_level=4
fig = gmaps.figure(map_type="TERRAIN",center=(48.210033,16.363449),zoom_level=3.5)

heat_layer = gmaps.heatmap_layer(locations, weights=aqi_20,dissipating=False,max_intensity=200,
                               point_radius=4)


info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>AQI 2020<dt><dd>{Avg AQI 2020}</dd>
</dl>
"""

city_info = [info_box_template.format(**row) for index, row in europe_df.iterrows()]
locations = europe_df[["Lat","Lng"]] 

symbols = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red',info_box_content=city_info)

                         
                         
fig.add_layer(heat_layer)
fig.add_layer(symbols)
fig


Figure(layout=FigureLayout(height='420px'))

## East

In [52]:
# Pulling the cities of the East for heatmap

eastern = cities_pd.loc[["Melbourne","Mumbai","New Dehli","Seoul","Wuhan"],
                              ["Avg AQI 2018","Avg AQI 2019","Avg AQI 2020","Lat","Lng"]]


In [53]:
# Reset Index
eastern = eastern.reset_index()

In [58]:
# Set locations
locations = eastern[["Lat","Lng"]]
# aqi_15 = eastern["Avg AQI 2015"].astype(float)
# aqi_16 = eastern["Avg AQI 2016"].astype(float)
# aqi_17 = eastern["Avg AQI 2017"].astype(float)
# aqi_18 = eastern["Avg AQI 2018"].astype(float)
# aqi_19 = eastern["Avg AQI 2019"].astype(float)   
aqi_20 = eastern["Avg AQI 2020"].astype(float)

In [59]:
# Heatmap for the Eastern Cities
#,center=(29.749907,-95.358421),zoom_level=4
fig = gmaps.figure(map_type="TERRAIN",center=(-12.4634,130.8456),zoom_level=2.25)

heat_layer = gmaps.heatmap_layer(locations, weights=aqi_20,dissipating=False,max_intensity=200,
                               point_radius=4)


info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>AQI 2020<dt><dd>{Avg AQI 2020}</dd>
</dl>
"""

city_info = [info_box_template.format(**row) for index, row in eastern.iterrows()]
locations = eastern[["Lat","Lng"]] 

symbols = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red',info_box_content=city_info)

                         
                         
fig.add_layer(heat_layer)
fig.add_layer(symbols)
fig

Figure(layout=FigureLayout(height='420px'))